In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import time 
from datetime import datetime, timedelta 

In [ ]:
# This notebook was used on Google Colab
youtube_data = pd.read_csv('/content/drive/My Drive/INvideos.csv') #reading csv for INDIA
youtube_data

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,kzwfHumJyYc,17.14.11,Sharry Mann: Cute Munda ( Song Teaser) | Parmi...,Lokdhun Punjabi,1,2017-11-12T12:20:39.000Z,"sharry mann|""sharry mann new song""|""sharry man...",1096327,33966,798,882,https://i.ytimg.com/vi/kzwfHumJyYc/default.jpg,False,False,False,Presenting Sharry Mann latest Punjabi Song Cu...
1,zUZ1z7FwLc8,17.14.11,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं...",HJ NEWS,25,2017-11-13T05:43:56.000Z,"पीरियड्स के समय|""पेट पर पति करता ऐसा""|""देखकर द...",590101,735,904,0,https://i.ytimg.com/vi/zUZ1z7FwLc8/default.jpg,True,False,False,"पीरियड्स के समय, पेट पर पति करता ऐसा, देखकर दं..."
2,10L1hZ9qa58,17.14.11,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,TFPC,24,2017-11-12T15:48:08.000Z,Stylish Star Allu Arjun @ ChaySam Wedding Rece...,473988,2011,243,149,https://i.ytimg.com/vi/10L1hZ9qa58/default.jpg,False,False,False,Watch Stylish Star Allu Arjun @ ChaySam Weddin...
3,N1vE8iiEg64,17.14.11,Eruma Saani | Tamil vs English,Eruma Saani,23,2017-11-12T07:08:48.000Z,"Eruma Saani|""Tamil Comedy Videos""|""Films""|""Mov...",1242680,70353,1624,2684,https://i.ytimg.com/vi/N1vE8iiEg64/default.jpg,False,False,False,This video showcases the difference between pe...
4,kJzGH0PVQHQ,17.14.11,why Samantha became EMOTIONAL @ Samantha naga ...,Filmylooks,24,2017-11-13T01:14:16.000Z,"Filmylooks|""latest news""|""telugu movies""|""telu...",464015,492,293,66,https://i.ytimg.com/vi/kJzGH0PVQHQ/default.jpg,False,False,False,why Samantha became EMOTIONAL @ Samantha naga ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37347,iNHecA3PJCo,18.14.06,फेकू आशिक़ - राजस्थान की सबसे शानदार कॉमेडी | ...,RDC Rajasthani,23,2018-06-13T08:01:11.000Z,"twinkle vaishnav comedy|""twinkle vaishnav""|""tw...",214378,3291,404,196,https://i.ytimg.com/vi/iNHecA3PJCo/default.jpg,False,False,False,PRG Music & RDC Rajasthani presents फेकू आशिक़...
37348,dpPmPbhcslM,18.14.06,Seetha | Flowers | Ep# 364,Flowers TV,24,2018-06-13T11:30:04.000Z,"flowers serials|""actress""|""malayalam serials""|...",406828,1726,478,1428,https://i.ytimg.com/vi/dpPmPbhcslM/default.jpg,False,False,False,"Flowers - A R Rahman Show,Book your Tickets He..."
37349,mV6aztP58f8,18.14.06,Bhramanam I Episode 87 - 12 June 2018 I Mazhav...,Mazhavil Manorama,24,2018-06-13T05:00:02.000Z,"mazhavil manorama|""bhramanam full episode""|""gt...",386319,1216,453,697,https://i.ytimg.com/vi/mV6aztP58f8/default.jpg,False,False,False,Subscribe to Mazhavil Manorama now for your da...
37350,qxqDNP1bDEw,18.14.06,Nua Bohu | Full Ep 285 | 13th June 2018 | Odia...,Tarang TV,24,2018-06-13T15:07:49.000Z,"tarang|""tarang tv""|""tarang tv online""|""tarang ...",130263,698,115,65,https://i.ytimg.com/vi/qxqDNP1bDEw/default.jpg,False,False,False,Nuabohu : Story of a rustic village girl who w...


In [ ]:
print(youtube_data.isnull().sum()) # no missing values except in description. Doesn't affect us as it is not used in our ML model
youtube_data.columns

video_id                    0
trending_date               0
title                       0
channel_title               0
category_id                 0
publish_time                0
tags                        0
views                       0
likes                       0
dislikes                    0
comment_count               0
thumbnail_link              0
comments_disabled           0
ratings_disabled            0
video_error_or_removed      0
description               561
dtype: int64


Index(['video_id', 'trending_date', 'title', 'channel_title', 'category_id',
       'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count',
       'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description'],
      dtype='object')

In [ ]:
trending_data = youtube_data.drop(columns=['title', 'channel_title', 'publish_time', 'tags', 'thumbnail_link', 'comments_disabled', 'ratings_disabled',
       'video_error_or_removed', 'description' ])
# Only keep columns that we are interested in
trending_data

,video_id,trending_date,category_id,views,likes,dislikes,comment_count
0,kzwfHumJyYc,17.14.11,1,1096327,33966,798,882
1,zUZ1z7FwLc8,17.14.11,25,590101,735,904,0
2,10L1hZ9qa58,17.14.11,24,473988,2011,243,149
3,N1vE8iiEg64,17.14.11,23,1242680,70353,1624,2684
4,kJzGH0PVQHQ,17.14.11,24,464015,492,293,66
...,...,...,...,...,...,...,...
37347,iNHecA3PJCo,18.14.06,23,214378,3291,404,196
37348,dpPmPbhcslM,18.14.06,24,406828,1726,478,1428
37349,mV6aztP58f8,18.14.06,24,386319,1216,453,697
37350,qxqDNP1bDEw,18.14.06,24,130263,698,115,65


In [ ]:
print(len(trending_data['video_id'].unique())) #16307 unique videos in original dataset

# change data format
trending_data['trending_date'] = pd.to_datetime(trending_data['trending_date'], format='%y.%d.%m')
trending_data['trending_date'] = [x.date() for x in trending_data['trending_date']]

# dates = trending_data['trending_date'].unique()

16307


In [ ]:
videos = trending_data['video_id'].unique()
print(len(videos))

final_df = pd.DataFrame(columns=['video_id','trending_date','category_id','views','likes','dislikes',	'comment_count',	'views_per_day',	'likes_per_day',	'dislikes_per_day',	'comments_per_day', 'trending'])


for video in range(len(videos)): # 16307 unique videos in original dataset
  vid_data = trending_data[trending_data['video_id'].isin([videos[video]])] # For each unique video, extract all of its entries from the dataset
  for index in range(len(vid_data)):
    vid_data = vid_data.reset_index(drop=True)
    if index < len(vid_data)-1:
      if vid_data.at[index+1, 'trending_date'] == vid_data.at[index, 'trending_date'] + timedelta(1):
         # if consecutive entries also have consecutive dates, then substract to find daily stats
        vid_data.at[index+1, 'views_per_day'] = vid_data.at[index+1, 'views'] - vid_data.at[index, 'views']
        vid_data.at[index+1, 'likes_per_day'] = vid_data.at[index+1, 'likes'] - vid_data.at[index, 'likes']
        vid_data.at[index+1, 'dislikes_per_day'] = vid_data.at[index+1, 'dislikes'] - vid_data.at[index, 'dislikes']
        vid_data.at[index+1, 'comments_per_day'] = vid_data.at[index+1, 'comment_count'] - vid_data.at[index, 'comment_count']

  final_df = final_df.append(vid_data, ignore_index=True)

final_df

16307


,video_id,trending_date,category_id,views,likes,dislikes,comment_count,views_per_day,likes_per_day,dislikes_per_day,comments_per_day,trending
0,kzwfHumJyYc,2017-11-14,1,1096327,33966,798,882,NaN,NaN,NaN,NaN,NaN
1,kzwfHumJyYc,2017-11-15,1,1651358,41670,1409,1545,555031.0,7704.0,611.0,663.0,NaN
2,kzwfHumJyYc,2017-11-16,1,2035734,46159,1807,2224,384376.0,4489.0,398.0,679.0,NaN
3,zUZ1z7FwLc8,2017-11-14,25,590101,735,904,0,NaN,NaN,NaN,NaN,NaN
4,zUZ1z7FwLc8,2017-11-15,25,1680175,1661,4078,0,1090074.0,926.0,3174.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
37347,AX_oTMHOb-E,2018-06-14,26,1849399,21875,1703,2238,NaN,NaN,NaN,NaN,NaN
37348,BgnpdzZFtzA,2018-06-14,23,29536,1559,64,94,NaN,NaN,NaN,NaN,NaN
37349,OsTbJ1rIo4U,2018-06-14,22,186197,855,86,25,NaN,NaN,NaN,NaN,NaN
37350,aKsKUk4RPLM,2018-06-14,24,90643,18060,165,4168,NaN,NaN,NaN,NaN,NaN


In [ ]:
nan_rows = final_df.isnull().sum()
print(nan_rows)
final_df = final_df[final_df['views_per_day'].notna()].reset_index(drop=True)

print(len(final_df.video_id.unique())) # 8418 unique videos remain in the dataset i.e. videos for which we could extract daily stats
final_df

video_id                0
trending_date           0
category_id             0
views                   0
likes                   0
dislikes                0
comment_count           0
views_per_day       21347
likes_per_day       21347
dislikes_per_day    21347
comments_per_day    21347
trending            37352
dtype: int64
8418


,video_id,trending_date,category_id,views,likes,dislikes,comment_count,views_per_day,likes_per_day,dislikes_per_day,comments_per_day,trending
0,kzwfHumJyYc,2017-11-15,1,1651358,41670,1409,1545,555031.0,7704.0,611.0,663.0,NaN
1,kzwfHumJyYc,2017-11-16,1,2035734,46159,1807,2224,384376.0,4489.0,398.0,679.0,NaN
2,zUZ1z7FwLc8,2017-11-15,25,1680175,1661,4078,0,1090074.0,926.0,3174.0,0.0,NaN
3,zUZ1z7FwLc8,2017-11-16,25,2180397,1986,5356,0,500222.0,325.0,1278.0,0.0,NaN
4,zUZ1z7FwLc8,2017-11-17,25,2485480,2216,6181,0,305083.0,230.0,825.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
16000,GwGLUH3IUdU,2018-06-14,25,145452,627,100,34,42668.0,137.0,26.0,14.0,NaN
16001,dEO0ezpwZng,2018-06-14,24,236723,673,195,73,68106.0,179.0,50.0,24.0,NaN
16002,0qAexDFUW2U,2018-06-14,24,100169,312,46,24,50812.0,88.0,25.0,11.0,NaN
16003,9ttVEPk2fhM,2018-06-14,23,212481,5769,204,585,72257.0,1086.0,44.0,117.0,NaN


In [ ]:
videos = final_df['video_id'].unique()
print(len(videos))

for video in range(len(videos)):
  vid_data = final_df[final_df['video_id'].isin([videos[video]])]#.reset_index(drop=True)
  for index in range(len(vid_data)):
    vid_data = vid_data.reset_index(drop=True)
    if index < len(vid_data)-1:
      if vid_data.at[index+1, 'trending_date'] == vid_data.at[index, 'trending_date'] + timedelta(1):
        # labelled 1 as video was on trending the next day
        vid_data.at[index, 'trending'] = 1
      else:
        # labelled 0 as video was not on trending the next day.
        vid_data.at[index, 'trending'] = 0
    else:
      #last entry is labelled 0 automatically as because video did not appear in trending later on
      vid_data.at[index, 'trending'] = 0

    current_vid = vid_data.at[index, 'video_id']
    current_date = vid_data.at[index, 'trending_date']

    temp_index = final_df[(final_df['video_id']==current_vid) & (final_df['trending_date']==current_date)].index
    final_df.at[temp_index, 'trending'] = vid_data.at[index, 'trending']

final_df

8418


,video_id,trending_date,category_id,views,likes,dislikes,comment_count,views_per_day,likes_per_day,dislikes_per_day,comments_per_day,trending
0,kzwfHumJyYc,2017-11-15,1,1651358,41670,1409,1545,555031.0,7704.0,611.0,663.0,1
1,kzwfHumJyYc,2017-11-16,1,2035734,46159,1807,2224,384376.0,4489.0,398.0,679.0,0
2,zUZ1z7FwLc8,2017-11-15,25,1680175,1661,4078,0,1090074.0,926.0,3174.0,0.0,1
3,zUZ1z7FwLc8,2017-11-16,25,2180397,1986,5356,0,500222.0,325.0,1278.0,0.0,1
4,zUZ1z7FwLc8,2017-11-17,25,2485480,2216,6181,0,305083.0,230.0,825.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
16000,GwGLUH3IUdU,2018-06-14,25,145452,627,100,34,42668.0,137.0,26.0,14.0,0
16001,dEO0ezpwZng,2018-06-14,24,236723,673,195,73,68106.0,179.0,50.0,24.0,0
16002,0qAexDFUW2U,2018-06-14,24,100169,312,46,24,50812.0,88.0,25.0,11.0,0
16003,9ttVEPk2fhM,2018-06-14,23,212481,5769,204,585,72257.0,1086.0,44.0,117.0,0


In [ ]:
# Manual verification
# print(trending_data[trending_data['video_id'] == 'j7I-d1B6HwE'])
# final_df[final_df['video_id'] == '7I-d1B6HwE']
# final_df[final_df['video_id'] == 'j7I-d1B6HwE']
# nan_rows = final_df.isnull().sum()
# print(nan_rows)

In [ ]:
# print(final_df[['category_id']].describe())
# final_df[final_df['trending'] == 0].count() # 5891
# final_df[final_df['trending'] == 1].count() # 28099 more 1s than 0s in dataset. it is imbalanced
# final_1s = pd.DataFrame(final_df[(final_df['trending'] == 1) & (final_df['category_id'] == 24)])
# final_0s = pd.DataFrame(final_df[(final_df['trending'] == 0) & (final_df['category_id'] == 24)])
# print(final_1s)
# print(final_0s)
# final_1s = final_1s.sample(frac=len(final_0s)/len(final_1s)).reset_index(drop=True)
# dataset = final_0s.append(final_1s,ignore_index=True).sample(frac=1).reset_index(drop=True)
# dataset

In [ ]:
print("Number of 0s in dataset:",len(final_df[final_df['trending'] == 0]))
print("Number of 1s in dataset:",len(final_df[final_df['trending'] == 1]))

final_df

Number of 0s in dataset: 8437
Number of 1s in dataset: 7568


,video_id,trending_date,category_id,views,likes,dislikes,comment_count,views_per_day,likes_per_day,dislikes_per_day,comments_per_day,trending
0,kzwfHumJyYc,2017-11-15,1,1651358,41670,1409,1545,555031.0,7704.0,611.0,663.0,1
1,kzwfHumJyYc,2017-11-16,1,2035734,46159,1807,2224,384376.0,4489.0,398.0,679.0,0
2,zUZ1z7FwLc8,2017-11-15,25,1680175,1661,4078,0,1090074.0,926.0,3174.0,0.0,1
3,zUZ1z7FwLc8,2017-11-16,25,2180397,1986,5356,0,500222.0,325.0,1278.0,0.0,1
4,zUZ1z7FwLc8,2017-11-17,25,2485480,2216,6181,0,305083.0,230.0,825.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
16000,GwGLUH3IUdU,2018-06-14,25,145452,627,100,34,42668.0,137.0,26.0,14.0,0
16001,dEO0ezpwZng,2018-06-14,24,236723,673,195,73,68106.0,179.0,50.0,24.0,0
16002,0qAexDFUW2U,2018-06-14,24,100169,312,46,24,50812.0,88.0,25.0,11.0,0
16003,9ttVEPk2fhM,2018-06-14,23,212481,5769,204,585,72257.0,1086.0,44.0,117.0,0


In [ ]:
# print(len(dataset[dataset['trending'] == 0]))
# print(len(dataset[dataset['trending'] == 1]))
# dataset

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score , precision_score , recall_score
from sklearn.preprocessing import StandardScaler

X = final_df[['views_per_day'	,'likes_per_day', 'dislikes_per_day',	'comments_per_day']]
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X) # standardize the features(columns) of X
Y = final_df['trending'].astype(int) 
# X = dataset[['views_per_day'	,'likes_per_day', 'dislikes_per_day',	'comments_per_day']]
# scaler = StandardScaler()
# scaler.fit(X)
# X = scaler.transform(X)
# # print(X)
# Y = dataset['trending'].astype(int)

# split data into train and test  randomly
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, shuffle=True)


model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
# print(model.predict_proba(X_test))
print(model.score(X_test, y_test))
# print(model.coef_)
print("Accuracy:",accuracy_score(y_test, y_pred))
print("Precision:",precision_score(y_test, y_pred))
print("Recall:",recall_score(y_test, y_pred))

0.6579210394802598
Accuracy: 0.6579210394802598
Precision: 0.7476547842401501
Recall: 0.4201370585134423
